In [1]:
#### Text classifcation model - Naive Bayes

In [2]:
import os 
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import *
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from itertools import islice
from sklearn.naive_bayes import MultinomialNB
import pickle

In [3]:
os.chdir("/home/visa/Python/News")

In [4]:
df = pd.read_csv("train.csv", header=None)

In [5]:
df.columns = ["Class","ShortDesc","Desc"]
df.head()

,Class,ShortDesc,Desc
0,3,Wall St. Bears Claw Back Into the Black (Reuters),"Reuters - Short-sellers, Wall Street's dwindli..."
1,3,Carlyle Looks Toward Commercial Aerospace (Reu...,Reuters - Private investment firm Carlyle Grou...
2,3,Oil and Economy Cloud Stocks' Outlook (Reuters),Reuters - Soaring crude prices plus worries\ab...
3,3,Iraq Halts Oil Exports from Main Southern Pipe...,Reuters - Authorities have halted oil export\f...
4,3,"Oil prices soar to all-time record, posing new...","AFP - Tearaway world oil prices, toppling reco..."


## Add a full list of stopwords

In [12]:
stops = stopwords.words('english')
ad_stopwords = ['monday', 'like', 'com', 'use', 'help', 'create', 'time', 'want', 'a', 'about', 'above', 'across', 'after', 'afterwards'] 
stops.extend(ad_stopwords)          

In [8]:
stops[0:10]

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', 'your']

In [8]:
##REGEX FOR DATE REMOVAL
#import re
txt='Monday, April, 4, 2016, 10:16:06'
re1='(Monday)'	# Day Of Week 1
re2='.*?'	# Non-greedy match on filler
re3='(April)'	# Month 1
re4='.*?'	# Non-greedy match on filler
re5='(4)'	# Day 1
re6='.*?'	# Non-greedy match on filler
re7='(2016)'	# Year 1
re8='.*?'	# Non-greedy match on filler
re9='(10:16:06)'	# HourMinuteSec 1

rg = re.compile(re1+re2+re3+re4+re5+re6+re7+re8+re9,re.IGNORECASE|re.DOTALL)
m = rg.search(txt)
if m:
    dayofweek1=m.group(1)
    month1=m.group(2)
    day1=m.group(3)
    year1=m.group(4)
    time1=m.group(5)

In [9]:
#import re
re1='(b)'	# Variable Name 1
re2='.*?'	# Non-greedy match on filler
re3='(4\\/04\\/2016)'	# DDMMYYYY 1

rd = re.compile(re1+re2+re3,re.IGNORECASE|re.DOTALL)
m = rg.search(txt)
if m:
    var1=m.group(1)
    ddmmyyyy1=m.group(2)

In [10]:
#import re
txt='\\\\r\\\\r____________________________\\\\r'

re1='(\\\\\\\\r\\\\\\\\r____________________________\\\\\\\\r)'	# Windows UNC 1

rt = re.compile(re1,re.IGNORECASE|re.DOTALL)
m = rg.search(txt)
if m:
    unc1=m.group(1)

In [6]:
#import nltk
def _remove_noise(input_text):
    input_text = str(input_text).encode('ascii', 'ignore')
    input_text = str(input_text).replace(",", "")
    #input_text = str(input_text).replace("\'\\", "")
    #input_text = str(input_text).replace("\'\", "")
    input_text = re.sub(rg, ' ', input_text)
    #input_text = re.sub([[:punct:]], '', input_text) -- this is a step in R 
    input_text = re.sub(rd, ' ', input_text)
    input_text = re.sub(rt, ' ', input_text)
    words = str(input_text).split()
    pos_words = nltk.pos_tag(words)
    noise_free_words = [i[0] for i in pos_words if i[1] in ('NN')]
    noise_free_words = [word for word in noise_free_words if word.lower() not in stops]
    return noise_free_words

In [13]:
df["cleaned"] = df.Desc.apply(_remove_noise)
df.head()

,Class,ShortDesc,Desc,cleaned
0,3,Wall St. Bears Claw Back Into the Black (Reuters),"Reuters - Short-sellers, Wall Street's dwindli...","[dwindling\\band, again.""]"
1,3,Carlyle Looks Toward Commercial Aerospace (Reu...,Reuters - Private investment firm Carlyle Grou...,"[investment, firm, reputation, defense, indust..."
2,3,Oil and Economy Cloud Stocks' Outlook (Reuters),Reuters - Soaring crude prices plus worries\ab...,"[crude, economy, outlook, stock, market, week,..."
3,3,Iraq Halts Oil Exports from Main Southern Pipe...,Reuters - Authorities have halted oil export\f...,"[oil, pipeline, after\\intelligence, militia, ..."
4,3,"Oil prices soar to all-time record, posing new...","AFP - Tearaway world oil prices, toppling reco...","[world, oil, toppling, menace, elections.']"


In [14]:
#from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
df['stemmed'] = df.cleaned.map(lambda x: ' '.join([lemmatizer.lemmatize(y) for y in x]))
df.stemmed.head()

0                              dwindling\\band again."
1    investment firm reputation defense industry pa...
2    crude economy outlook stock market week depth ...
3    oil pipeline after\\intelligence militia oil o...
4                world oil toppling menace elections.'
Name: stemmed, dtype: object

In [15]:
cvec = CountVectorizer(stop_words= stops, min_df=1, max_df=.5, ngram_range=(1,2))


In [16]:
#from itertools import islice
cvec.fit(df.stemmed)

list(islice(cvec.vocabulary_.items(), 20))


[('rival defeat', 242802),
 ('quetta bomb', 226830),
 ('key system', 150503),
 ('ailment', 8831),
 ('week opposition', 317228),
 ('oil broker', 192668),
 ('marriage partner', 168510),
 ('hsh', 135454),
 ('attack statement', 18689),
 ('circuit government', 51165),
 ('year onboard', 325068),
 ('depart rebel', 80348),
 ('wake racy', 312566),
 ('search none', 251509),
 ('feature test', 104445),
 ('money target', 177663),
 ('event season', 98177),
 ('siebel week', 260949),
 ('story court', 276725),
 ('spokesman technique', 269864)]

In [17]:
len(cvec.vocabulary_)

328294

In [18]:
cvec = CountVectorizer(stop_words=stops, min_df=.001, max_df=.99, ngram_range=(1,2))
cvec.fit(df.stemmed)
len(cvec.vocabulary_)
#pickle.dump(cvec.vocabulary_,open("feature.pkl","wb"))

1098

In [19]:
cvec_counts = cvec.transform(df.stemmed)
print ('sparse matrix shape:', cvec_counts.shape)
print ('nonzero count:', cvec_counts.nnz)
print ('sparsity: %.2f%%' % (100.0 * cvec_counts.nnz / (cvec_counts.shape[0] * cvec_counts.shape[1])))

sparse matrix shape: (120000, 1098)
nonzero count: 491545
sparsity: 0.37%


In [20]:
transformer = TfidfTransformer()
transformed_weights = transformer.fit_transform(cvec_counts)
transformed_weights

#transformer = TfidfTransformer()
#loaded_vec = CountVectorizer(decode_error="replace",vocabulary=pickle.load(open("feature.pkl", "rb")))
#tfidf = transformer.fit_transform(loaded_vec.fit_transform(cvec_counts))

<120000x1098 sparse matrix of type '<class 'numpy.float64'>'
	with 491545 stored elements in Compressed Sparse Row format>

In [21]:
weights = np.asarray(transformed_weights.mean(axis=0)).ravel().tolist()
weights_df = pd.DataFrame({'term': cvec.get_feature_names(), 'weight': weights})
weights_df.sort_values(by='weight', ascending=False).head(20)

,term,weight
197,company,0.017661
1095,yesterday,0.016329
1092,year,0.015628
1068,week,0.013733
764,quot,0.013155
1085,world,0.013009
993,today,0.012786
404,government,0.012462
616,night,0.012041
391,game,0.012005


In [23]:
target = df["Class"]

In [22]:
nb = MultinomialNB()

In [24]:
nb.fit(transformed_weights, target)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [25]:
df1 = pd.read_csv("test.csv", header=None)

In [26]:
df1.columns = ["Class","ShortDesc","Desc"]
df1.head()

,Class,ShortDesc,Desc
0,3,Fears for T N pension after talks,Unions representing workers at Turner Newall...
1,4,The Race is On: Second Private Team Sets Launc...,"SPACE.com - TORONTO, Canada -- A second\team o..."
2,4,Ky. Company Wins Grant to Study Peptides (AP),AP - A company founded by a chemistry research...
3,4,Prediction Unit Helps Forecast Wildfires (AP),AP - It's barely dawn when Mike Fitzpatrick st...
4,4,Calif. Aims to Limit Farm-Related Smog (AP),AP - Southern California's smog-fighting agenc...


In [27]:
df1["cleaned"] = df1.Desc.apply(_remove_noise)
df1.head()

,Class,ShortDesc,Desc,cleaned
0,3,Fears for T N pension after talks,Unions representing workers at Turner Newall...,"[parent, firm]"
1,4,The Race is On: Second Private Team Sets Launc...,"SPACE.com - TORONTO, Canada -- A second\team o...","[second\\team, contest, space, flight, first\\..."
2,4,Ky. Company Wins Grant to Study Peptides (AP),AP - A company founded by a chemistry research...,"[company, chemistry, researcher, grant, method..."
3,4,Prediction Unit Helps Forecast Wildfires (AP),AP - It's barely dawn when Mike Fitzpatrick st...,"[dawn, shift, blur, day]"
4,4,Calif. Aims to Limit Farm-Related Smog (AP),AP - Southern California's smog-fighting agenc...,"[smog-fighting, agency, variety, nation's, air..."


In [28]:
#from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
df1['stemmed'] = df1.cleaned.map(lambda x: ' '.join([lemmatizer.lemmatize(y) for y in x]))
df1.stemmed.head()

0                                          parent firm
1    second\\team contest space flight first\\launc...
2    company chemistry researcher grant method amin...
3                                  dawn shift blur day
4    smog-fighting agency variety nation's air poll...
Name: stemmed, dtype: object

In [29]:
cvec1 = CountVectorizer(stop_words= stops, min_df=0.001, max_df=0.9, ngram_range=(1,2))

In [30]:
#from itertools import islice
cvec1.fit(df1.stemmed)
list(islice(cvec1.vocabulary_.items(), 20))

[('hardware', 457),
 ('ban', 81),
 ('infotype', 502),
 ('auction', 68),
 ('candidate', 143),
 ('father', 385),
 ('gun', 451),
 ('partner', 720),
 ('prescription', 777),
 ('deficit', 293),
 ('oracle', 697),
 ('charge', 174),
 ('cooperation', 242),
 ('ago', 25),
 ('agreement', 26),
 ('worth', 1169),
 ('way', 1140),
 ('someone', 958),
 ('equipment maker', 360),
 ('distance', 316)]

In [31]:
cvec1 = CountVectorizer(stop_words=stops, min_df=.001, max_df=.8, ngram_range=(1,2))
cvec1.fit(df1.stemmed)
len(cvec1.vocabulary_)

1176

In [32]:
cvec_counts1 = cvec.transform(df1.stemmed)
print ('sparse matrix shape:', cvec_counts1.shape)
print ('nonzero count:', cvec_counts1.nnz)
print ('sparsity: %.2f%%' % (100.0 * cvec_counts1.nnz / (cvec_counts1.shape[0] * cvec_counts1.shape[1])))

sparse matrix shape: (7600, 1098)
nonzero count: 30709
sparsity: 0.37%


In [33]:
transformer1 = TfidfTransformer()
transformed_weights1 = transformer.fit_transform(cvec_counts1)
transformed_weights1

<7600x1098 sparse matrix of type '<class 'numpy.float64'>'
	with 30709 stored elements in Compressed Sparse Row format>

In [34]:
preds = nb.predict(transformed_weights1)

In [35]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

In [36]:
confusion_matrix(df1["Class"], preds)

array([[1501,  145,  159,   95],
       [ 180, 1594,   58,   68],
       [ 205,   69, 1373,  253],
       [ 228,  107,  251, 1314]])

In [37]:
accuracy_score(df1["Class"], preds, normalize=True)

0.7607894736842106